In [1]:
import numpy as np
import pandas as pd
import pyvista as pv
import vtk 
#pv.set_jupyter_backend('pythreejs')  

In [2]:
%%bash 
docker cp physx41_container:/DUMMY/GetGeometry/BUILD/simulation.dat .
pwd
ls simulation.dat

/Users/poderozita/z2022_1/REPOSITORY/NVCC
simulation.dat


In [3]:
%%time 

def GetCubeSource(length, center):
    c = vtk.vtkCubeSource() 
    c.SetCenter(center)
    c.SetXLength(length)
    c.SetYLength(length)
    c.SetZLength(length)
    c.Update()
    return c.GetOutputDataObject(0) #vtkPolyData


def GetCubeSource(length, center):
    cube = pv.Cube(center=center,
                        x_length=length,
                        y_length=length,
                        z_length=length
                  )
    return cube #vtkPolyData


def AppendVtps( vtkObjList ) :
  appendPolyData = vtk.vtkAppendPolyData()

  for vtkObj in vtkObjList : appendPolyData.AddInputData(vtkObj)
  appendPolyData.Update()

  cleaned = vtk.vtkCleanPolyData()
  cleaned.SetInputConnection(appendPolyData.GetOutputPort());
  cleaned.Update()
  return cleaned.GetOutputDataObject(0)


def GetRotation(vtkObj, angle, axis): # angle is in degrees
  transform = vtk.vtkTransform()
  transform.RotateWXYZ(angle, axis)

  tf = vtk.vtkTransformFilter()
  tf.SetInputData(vtkObj)
  tf.SetTransform(transform)
  tf.Update()
  return tf.GetOutputDataObject(0)


def GetQRotation(vtkobj, q) :
    Q = vtk.vtkQuaterniond(q)
    axis = np.full(3,np.nan); 
    angle = Q.GetRotationAngleAndAxis(axis) # angle (in radians)
    angle = np.rad2deg(angle)
    rotated = GetRotation(vtkobj, angle, axis)
    return rotated


def GetDisplace(vtkObj, Translate):
  transform = vtk.vtkTransform()
  transform.Translate(Translate)

  tf = vtk.vtkTransformFilter()
  tf.SetInputData(vtkObj)
  tf.SetTransform(transform)
  tf.Update()
  return tf.GetOutputDataObject(0)


class PhysX :
    
    def __init__(self): 
        return 
    

    def SetDataSimulation(self, fname):
        self.df = pd.read_csv(fname, sep=" ",header=0)
        self.dfGroupBy = self.df.groupby("istep")

        g = self.dfGroupBy.get_group(70)
        vtp = self.Groups2VTK(g) 
        
        Vtps = [self.Groups2VTK(v) for k,v in self.dfGroupBy]
        return Vtps 
    
    
    def Groups2VTK(self, group):
        quaternions = group[["qx","qy","qz","qw"]].to_numpy() # qw [rad]
        centers = group[["x","y","z"]].to_numpy()
        assert len(centers) == len(quaternions)

        cubes = [GetCubeSource(4.0,(0,0,0)) for c in centers]
        cubes = [GetQRotation(c,q) for c,q in zip(cubes,quaternions)]        
        cubes = [GetDisplace(c,q) for c,q in zip(cubes,centers)]        
        Vtp = AppendVtps(cubes)
        return Vtp
        
    
fin="/Users/poderozita/z2022_1/REPOSITORY/NVCC/simulation.dat"

px = PhysX()
Vtps = px.SetDataSimulation(fin) 

CPU times: user 4min 53s, sys: 6.51 s, total: 5min
Wall time: 6min 27s


### Interactive 

In [4]:
plane = pv.Plane(i_size=100,j_size=100, direction=(0,1,0))

pl = pv.Plotter()
pl.add_mesh(plane, color="tan", show_edges=True,lighting=False)
pl.add_mesh(Vtps[1],show_edges=True,color='r',line_width=1,lighting=False)
pl.add_mesh(Vtps[-1],show_edges=True,color='g',line_width=1,lighting=False)

pl.set_background('black')
pl.set_position([-150,50,200])
pl.set_focus([0.0,20,24]) 
pl.set_viewup((0.0, 1.0, 0.0))

pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Git

In [5]:
filename = "cubes"

mesh = pv.PolyData(Vtps[0])

plotter = pv.Plotter()
#plotter.open_movie(filename+".mp4")
plotter.open_gif(filename+".gif")

plane = pv.Plane(i_size=100,j_size=100, direction=(0,1,0))
plotter.add_mesh(plane, color="tan", show_edges=True,lighting=False)
plotter.add_mesh(mesh, lighting=False, show_edges=True, color='g')
plotter.add_mesh(mesh.outline_corners())

plotter.set_background('black')
plotter.set_position([-150,50,200])
plotter.set_focus([0.0,20,24]) 
plotter.set_viewup((0.0, 1.0, 0.0))

plotter.show(auto_close=False)  
plotter.write_frame() 
for i,vtp in enumerate(Vtps[1:]):
    vtp = pv.PolyData(vtp)
    mesh.points = vtp.points 
    plotter.add_text(f"step:{i}", name='time-label',font_size=6,color="black")
    plotter.write_frame() 

plotter.close()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [6]:
for i,vtp in enumerate(Vtps):
    mesh = pv.PolyData(vtp)
    mesh.save("time%03d.vtp"%i)

## Appendix
### Transformations

In [7]:
cube = pv.Cube(center=(0,0,0),z_length=2)

axis = [0.0,0.0,1.0]
axis = np.array(axis)
rotated = GetRotation(cube, 45.0, axis) 

center = (2,0,0)
center = np.array(center)
displaced = GetDisplace(rotated,center) 


pl = pv.Plotter()
pl.add_mesh(cube,show_edges=True,color='r',line_width=3) 
pl.add_mesh(rotated,show_edges=True,color='b',line_width=3) 
pl.add_mesh(displaced,show_edges=True,color='g',line_width=3) 
pl.add_axes_at_origin(zlabel='')
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [8]:
%%bash 
uname -a 
date
pwd

Darwin Air-de-j.lan 20.2.0 Darwin Kernel Version 20.2.0: Wed Dec  2 20:39:59 PST 2020; root:xnu-7195.60.75~1/RELEASE_X86_64 x86_64
Thu May 12 11:21:48 CEST 2022
/Users/poderozita/z2022_1/REPOSITORY/NVCC
